In [1]:
# PY libraries
import pandas as pd
import re
import nltk

# basic funcs
"""
def default_steam():
    steam.drop('Unnamed: 0', inplace=True, axis=1)
    steam = steam.sort_values("app_id")

def letters_count_update():
    steam['letters_count'] = steam['review_text'].apply(lambda x: len(x))

def comment_filter():
    steam = steam[steam['letters_count'] > 10]
    steam = steam[(steam['letters_count'] >= 25) | (steam['review_votes'] == 1)]
    steam = steam.sort_values('letters_count')

def minus_to_1():
    for i in steam.index:
        if steam.loc[i, "review_score"] == -1:
            steam.loc[i, "review_score"] = 0
"""

In [ ]:
# NULLS TYPES LETTERS_COUNT COMMENT_FILTER
steam = pd.read_csv(r'SteamReviews.csv')

# int type
# print(steam.isnull().sum(axis = 0))
steam = steam.astype({'app_id':'int', 'review_score':'int', 'review_votes':'int'})

# NULL
steam.dropna(inplace = True)

# -1 to 0
for i in steam.index:
        if steam.loc[i, "review_score"] == -1:
            steam.loc[i, "review_score"] = 0

# letters counts column
steam['letters_count'] = steam['review_text'].apply(lambda x: len(x))

# comments
steam = steam[steam['letters_count'] > 10]
steam = steam[(steam['letters_count'] >= 25) | (steam['review_votes'] == 1)]
steam = steam.sort_values('letters_count')

In [ ]:
### LOAD FILE_1
# steam.to_csv('steam1.csv')
steam = pd.read_csv(r'steam1.csv')
steam.drop('Unnamed: 0', inplace=True, axis=1)
steam = steam.sort_values("app_id")
steam.head()

In [ ]:
# LOWER REGEX COMMENT_FILTER
def cleanSymbols(text):
    res = re.sub(r"[^a-zA-Z\s]+", "", text)
    return res

def cleanLinks(text):
    res = re.sub(r"http\s+", "", text)
    res = re.sub("w+://s+", "", res)
    return res
steam['review_text'] = steam['review_text'].astype(str).str.lower()
steam['review_text'] = steam['review_text'].apply(cleanLinks)
steam['review_text'] = steam['review_text'].apply(cleanSymbols)

# UPDATE LETTERS_COUNT & DELETE COMMENTS
steam['letters_count'] = steam['review_text'].apply(lambda x: len(x))
steam = steam.sort_values('letters_count')
steam = steam[steam['letters_count'] > 10]
steam = steam[(steam['letters_count'] >= 25) | (steam['review_votes'] == 1)]
steam = steam.sort_values('letters_count')

In [ ]:
### LOAD FILE_2 4988035
# steam.to_csv('steam2.csv')
steam = pd.read_csv(r'steam2.csv')
steam.drop('Unnamed: 0', inplace=True, axis=1)
steam = steam.sort_values("app_id")
steam.head()

In [ ]:
# TOKENS
def delete_long_and_tokenize(text):
    res = [w for w in text.split() if len(w) < 20]
    if len(res) <= 3:
        res = ''
    return res
steam["tokens"] = steam["review_text"].apply(lambda x: delete_long_and_tokenize(x))
steam = steam[steam['tokens'].str.len() > 0]

In [ ]:
### LOAD FILE_3 TOKENIZED 4787150
# steam.to_csv('steam3.csv')
steam = pd.read_csv(r'steam3.csv')
steam.drop('Unnamed: 0', inplace=True, axis=1)
steam = steam.sort_values("app_id")
steam.head()

In [ ]:
# STOPWORDS
# nltk.download('stopwords')
SW = list(nltk.corpus.stopwords.words("english"))
for i in range(len(SW)):
    SW[i] = re.sub(r"[^a-zA-Z]+", '', SW[i])
stop_words = set(SW)

def stopwords_cleaner(text):
    text = str(text)[2:-2]
    text = text.split("', '")
    res = [w for w in text if w not in stop_words]
    if len(res) <= 4:
        return ''
    return ' '.join(res)
steam['tokens'] = steam['tokens'].apply(stopwords_cleaner)
steam = steam[steam['tokens'].str.len() > 0]

steam.head()

In [18]:
### LOAD FILE_4 STOPWORDS 4333260
# steam.to_csv('steam4.csv')
steam = pd.read_csv(r'steam4.csv')
steam.drop('Unnamed: 0', inplace=True, axis=1)
steam = steam.sort_values("app_id")
steam.head()

,app_id,app_name,review_text,review_score,review_votes,letters_count,tokens
0,10,Counter-Strike,best shooter ever buy it or regret it,1,0,37,best shooter ever buy regret
3671,10,Counter-Strike,ahh csfor many of us born in the s this game t...,1,0,196,ahh csfor many us born game took fps virginity...
3670,10,Counter-Strike,this game started the fps franchise well techn...,1,0,93,game started fps franchise well technically st...
3669,10,Counter-Strike,epicness ummm you might get addicted xd this g...,1,1,71,epicness ummm might get addicted xd game aweso...
3668,10,Counter-Strike,you can already get it free on the offical web...,0,1,50,already get free offical website


In [29]:
sub = steam[steam['app_id'] == 10]
sub.head()

,app_id,app_name,review_text,review_score,review_votes,letters_count,tokens
0,10,Counter-Strike,best shooter ever buy it or regret it,1,0,37,best shooter ever buy regret
3671,10,Counter-Strike,ahh csfor many of us born in the s this game t...,1,0,196,ahh csfor many us born game took fps virginity...
3670,10,Counter-Strike,this game started the fps franchise well techn...,1,0,93,game started fps franchise well technically st...
3669,10,Counter-Strike,epicness ummm you might get addicted xd this g...,1,1,71,epicness ummm might get addicted xd game aweso...
3668,10,Counter-Strike,you can already get it free on the offical web...,0,1,50,already get free offical website


In [30]:
# LEMMAS
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
#nltk.download('wordnet')
#nltk.download('omw-1.4')

lem = WordNetLemmatizer()

def get_pos(w):
    tup = nltk.pos_tag(w)
    if tup[1].startswith('N'):
        return wordnet.NOUN
    elif tup[1].startswith('V'):
        return wordnet.VERB
    elif tup[1].startswith('J'):
        return wordnet.ADJ
    elif tup[1].startswith('R'):
        return wordnet.ADV
    else:
        return ''

def lem_tokens(text):
    text = text.split()
    lemmas = ' '.join([lem.lemmatize(w, get_pos(w)) for w in text])
    return lemmas

sub['lemmas'] = sub['tokens'].apply(lem_tokens)
sub.head()

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle[0m

  Searched in:
    - 'C:\\Users\\Milka/nltk_data'
    - 'c:\\Users\\Milka\\AppData\\Local\\Programs\\Python\\Python310\\nltk_data'
    - 'c:\\Users\\Milka\\AppData\\Local\\Programs\\Python\\Python310\\share\\nltk_data'
    - 'c:\\Users\\Milka\\AppData\\Local\\Programs\\Python\\Python310\\lib\\nltk_data'
    - 'C:\\Users\\Milka\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
# Determine the correlation using a heatmap to visualize the data
plt.figure(figsize=(20, 10))
sns.heatmap(dataset.corr(), annot=True, cmap='Greens')
plt.show()

In [ ]:
# Drop multicollinear feature
dataset = dataset.drop('share', axis=1)

# Drop features with very low correlation
dataset = dataset.drop(['age', 'months'], axis=1)

# Plot heamap
plt.figure(figsize= (20, 10))
sns.heatmap(dataset.corr(), annot=True, cmap='Greens')
plt.show()

In [ ]:
# Split dataset
X = dataset.drop('card', axis=1)
y = dataset['card']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=100)

X_train.head()

In [ ]:
# Scaling
scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Try different parameters
params = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

logistic_model = GridSearchCV(
    estimator=LogisticRegression(random_state=0, multi_class='ovr'),
    param_grid=params,
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Train Logistic Regression
logistic_model.fit(X_train, y_train)

# Making predictions
logistic_predictions = logistic_model.predict(X_test)

# Calculate accuracy and print result
accuracy = accuracy_score(logistic_predictions, y_test) * 100
print('Accuracy of the model is {:.2f}'.format(accuracy))
print(logistic_model.best_params_)

# Display confusion matrix
cm = confusion_matrix(y_test, logistic_predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

plt.savefig('logistic_confusion.png', dpi=300)

In [ ]:
# Try different parameters
params = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

svm_model = GridSearchCV(
    estimator=svm.LinearSVC(random_state=100, max_iter=1000000),
    param_grid=params,
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Train SVM 
svm_model.fit(X_train, y_train)

# Make predictions
svm_prediction = svm_model.predict(X_test)

# Calculate accuracy and print result
accuracy = accuracy_score(svm_prediction, y_test) * 100
print('Accuracy of the model is {:.2f}'.format(accuracy))
print(svm_model.best_params_)

# Display confusion matrix
cm = confusion_matrix(y_test, svm_prediction)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

plt.savefig('svm_confusion.png', dpi=300)

In [ ]:
# Try different parameters
params = {
    'n_neighbors':  range(1, 15, 2),
    'p': [1, 2],
    'weights': ['uniform', 'distance']
}


knn_model = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=params,
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Train KNN
knn_model.fit(X_train, y_train)

# Make prediction
knn_prediction = knn_model.predict(X_test)

# Calculate accuracy and print result
accuracy = accuracy_score(knn_prediction, y_test) * 100
print('Accuracy of the model is {:.2f}'.format(accuracy))
print(knn_model.best_params_)

# Display confusion matrix
cm = confusion_matrix(y_test, knn_prediction)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

plt.savefig('knn_confusion.png', dpi=300)